# Ayudantía 07 - Threading

__Autores: Christian Eilers (@tatanpoker) y Dante Pinto (@drpinto1)__


# Recuerda responder el [Feedback](https://forms.gle/vAz7o3Etsh427bKv5) :D

### ¿Qué es un Thread?
_Una secuencia muy puequeña, de tareas encadenadas, que puede ser ejecutada por un sistema operativo, eso significa, básicamente, que **un thread es solamente, una sección de código**._

### ¿En qué se diferencia eso de lo que hemos hecho hasta ahora?
_Hasta ahora, sin saberlo, han estado usando threads; siendo más específicos, han estado usando **un thread**, la materia de esta semana tiene como objetivo enseñarles a manejar **múltiples threads**._

### ¿Para qué quiero manejar múltiples threads?
_La gran ventaja que brinda utilizar múltiples threads es la habilidad de ejecutarlos en ~~pseudo~~paralelo_.
_La mejor manera de visualizar lo anterior es a través de un juego._

# Juego básico
## Los enemigos
Imaginemos que queremos modelar a un enemigo en un juego, que cambia su posición cada 1 segundo. Esto lo podemos hacer más o menos así:

In [ ]:
import time
import random

class Enemigo:
    def __init__(self, nombre, x, y):
        self.x = x
        self.y = y
        self.nombre = nombre
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}\n')
        

el_malo = Enemigo('el_malo', 0, 0)

for _ in range(5):
    el_malo.mover()
    print()
    time.sleep(1)

Extendiéndolo a múltiples enemigos, tenemos algo así:

In [ ]:
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_mas_malo = Enemigo('el_mas_malo', 0, 0)

for _ in range(3):
    el_malo.mover()
    el_muy_malo.mover()
    el_mas_malo.mover()
    print()
    time.sleep(1)

## El Jugador
Si ahora queremos implementar al jugador, podemos seguir la misma lógica, recibiendo la acción del usuario a través de la sentencia `input`

In [ ]:
class Jugador:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def mover(self):
        mov = int(input('Seleccione un número de casillas para mover. '))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}\n')

jugador = Jugador(0, 0)
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_mas_malo = Enemigo('el_mas_malo', 0, 0)

for _ in range(3):
    jugador.mover()
    el_malo.mover()
    el_muy_malo.mover()
    el_mas_malo.mover()
    print()
    time.sleep(1)

Es claro que el comportamiento anterior no es el esperado, pues los enemigos solo se mueven luego de que se mueva el jugador, no cada un segundo. Podemos arreglar lo anterior con ayuda de threads.

### La sintaxis básica

In [ ]:
import threading as thr

def thredeador():
    wait= random.randint(1, 5)
    mi_thread = thr.current_thread()
    print(f'Hola, soy el thread {mi_thread.name}, esperare {wait}s \n')
    time.sleep(wait)
    print(f'Thread {mi_thread.name} terminando x__x \n')

uno = thr.Thread(name='thread uno', target=thredeador)
dos = thr.Thread(name='thread dos', target=thredeador)

uno.start()
dos.start()

## Aplicándolo al juego

### IMPORTANTE:
El siguiente bloque de código **va a fallar** luego de ingresar el primer input, esto se debe a la ~~mala~~ manera que tiene jupyter para manejar los threads.

Si copias este mismo código a un archivo .py y lo ejecutas como a cualquier otr programa, va a funcionar como se espera.

In [ ]:
def movedor(movible):  # duck typing ;)
    for _ in range(3):
        movible.mover()
        time.sleep(1)

el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(0, 0)


mueve0 = thr.Thread(target=movedor, args=(jugador,))
mueve1 = thr.Thread(target=movedor, args=(el_malo,))
mueve2 = thr.Thread(target=movedor, args=(el_muy_malo,))
mueve3 = thr.Thread(target=movedor, args=(el_mas_malo,))

mueve_todo = [mueve0, mueve1, mueve2, mueve3]
for mueve in mueve_todo:
    mueve.start()

## Threading + OOP
Podemos hacer lo mismo que en el bloque anterior aplicando una parte de la materia de OOP, nuestra gran amiga Herencia.

In [ ]:
class Enemigo(thr.Thread):
    def __init__(self, nombre, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.nombre = nombre
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}\n')
        
    def run(self):
        for i in range(3):
            self.mover()
            time.sleep(1)

class Jugador(thr.Thread):
    def __init__(self, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y

    def mover(self):
        mov = int(input('Seleccione un número de casillas para mover.'))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}\n')
        
    def run(self):
        for i in range(3):
            self.mover()
            time.sleep(1)

el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

##  Game Over
¿Qué pasa si queremos que el programa termine luego de que el jugador se mueva solamente una vez?
Redefinamos al jugador para que solamente se pueda mover una vez

In [ ]:
class Jugador(thr.Thread):
    def __init__(self, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y

    def mover(self):
        mov = int(input('Seleccione un número de casillas para mover.'))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}\n')
        
    def run(self):
        for i in range(1):
            self.mover()

el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

Nuevamente, lo anterior no realiza lo que deseamos, luego de que el jugador se mueve, los enemigos siguien funcionando y el programa no de tiene su ejecución, lo que nos plantea la __gran desventaja__ que tiene threading. La sincronización.

### Daemon Thread
La primera herramienta de sincronización que utilizaremos serán lo _daemon threads_; normalmente, el programa continúa su ejecución hasta que todos sus threads terminaron, sin embargo, los daemon threads no impiden la finalización del programa, que es exactamente lo que queremos para los enemigos.

Existen dos maneras de crear daemon threads.

#### Manera 1:
Sin alterar la definición de las clases

In [ ]:
# Misma instanciación
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

# Agregamos las lineas convirtiendo los threads
el_malo.daemon = True
el_muy_malo.daemon = True
el_malisimo.daemon = True

# Iniciamos
el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

#### Manera 2:
Declarándolo en la definición de la clase.

In [ ]:
# Esta definción es exactamente la misma
class Enemigo(thr.Thread):
    def __init__(self, nombre, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.nombre = nombre
        self.daemon = True  # Excepto por esta linea, donde decalramos al thread como Daemon
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}\n')
        
    def run(self):
        for i in range(3):
            self.mover()
            time.sleep(1)
            
# Instanciamos
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

# Iniciamos
el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

## Creación de enemigos
Asumamos ahora que, al igual que el jugador, cada vez que un enemigo deja de moverse, muere, pero no queremos que el juego se quede sin enemigos, o sea, cada vez que mueran los 3 enemigos del juego, queremos crear 3 enemigos nuevos. 

La manera intuitiva de hacer lo anterior sería con un loop que esté constantemente revisando si los enemigos están vivos o no, sin embargo esto es una __muy mala práctica__ que se conoce como _Busy Waiting_, pues estamos activamente ejecutando un proceso, consumiendo una gran cantidad de recursos, para simplemente esperar a la finalización de otros procesos, en este caso, los enemigos.


### threading.Thread.join()
La segunda herramienta de sincronización que utilizaremos será el método `join`. Este método __detiene por completo la ejecución del programa principal__, hasta que el thread cuyo join fue llamado termina de ejecutarse.

Para realizar lo anterior usando join, debemos hacer los siguiente:

In [ ]:
# Instanciamos e inicializamos al jugador
jugador = Jugador(5, 2)
jugador.start()

c = 0
while True:
    # Instanciamos a los enemigos
    el_malo = Enemigo(f'el_malo {c}', 0, 0)
    el_muy_malo = Enemigo(f'el_muy_malo {c}', 0, 0)
    el_malisimo = Enemigo(f'el_malisimo {c}', 0, 0)
    
    c += 1
    
    # Inicializamos a los enemigos
    el_malo.start()
    el_muy_malo.start()
    el_malisimo.start()
    
    # Hacemos join de los enemigos
    el_malo.join()
    el_muy_malo.join()
    el_malisimo.join()

## Inicio del juego

Hasta ahora, nuestro juego ha estado iniciando de manera automática al ejecutar el programa, son embargo, no siempre es lo que queremos que ocurra. ¿Cómo hacemos que los enemigos no puedan moverse hasta que se mueva el jugador por primera vez?


### threading.Event()
La tercera herramienta de sincronización que utilizaremos será la clase `Event` del módulo threading. Los Eventos de threading son objetos que funcionan de manera similar al join, sin embargo son mucho más versátiles, pues tiene la capacidad de __detener por completo la ejecucion de cualquier thread__ hasta que cierta condición se cumpla. Además de lo anterior, un mismo evento puede detener (o reanudar) la ejecución de __múltiples threads a la vez__.

La manera de hacer lo anterior es a través de los métodos `wait`, `set` y `clear`, aunque para nuestro juego solo necesitamos los primeros dos.

In [ ]:
class Enemigo(thr.Thread):
    def __init__(self, nombre, x, y, start_event, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.nombre = nombre
        self.start_event = start_event  # Agregamos el evento
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}\n')
        
    def run(self):
        self.start_event.wait()  # Esperamos que ocurra el evento antes de funcionar
        for i in range(3):
            self.mover()
            time.sleep(1)
            
            
class Jugador(thr.Thread):
    def __init__(self, x, y, move_event, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.move_event = move_event

    def mover(self):
        mov = int(input('Seleccione un número de casillas para mover.'))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}\n')
        self.move_event.set() # El jugador se movió, por lo que activamos el evento
        
    def run(self):
        for i in range(1):
            self.mover()
            time.sleep(1)
            
# Creamos el evento
pl_move = thr.Event()

# Instanciamos
el_malo = Enemigo('el_malo', 0, 0, pl_move)
el_muy_malo = Enemigo('el_muy_malo', 0, 0, pl_move)
el_malisimo = Enemigo('el_malisimo', 0, 0, pl_move)
jugador = Jugador(5, 2, pl_move)

# Iniciamos
el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

## Move Count
Finalmente, para futuros estudios, queremos registrar cuantos movimientos realizan los enemigos antes de que se termine el juego, la manera más simple (y obvia) sería guardar un contador de movimientos y cada vez que un enemigo decida moverse, que le sume uno al contador; sin embargo al hacer esto, nos encontramos con el problema, posiblemente, más grande de la sincronización de threads, el compartir recursos.

Al ejecutar código de manera pseudoparalela, no tenemos manera de saber en qué orden ocurriran las cosas, puesto que, al momento de hacerse la traducción de código a órdenes para el computador, python decide pausar la ejecución de un thread y reanudar la de otro de manera arbitraria, por lo que los threads __no se ejecutarán en el orden en el que se llamó a su start y no necesariamente completarán la ejecución de sus funciones antes de que otro thread lo haga__.

### threading.Lock()
La última herramienta de sincronización que utilizaremos será la clase `Lock` del módulo threading. Los locks de python son objetos que pueden ser entregados a los threads y que detienen la ejecución de estos si es que algún otro está haciendo uso de ellos.

El siguiente ejemplo, independiente de nuestro juego, nos permite ver el problema explicado anteriormente y su solución.

In [ ]:
class TicketList:
    def __init__(self, t_number):
        self.tickets = [1 for i in range(t_number)] # creamos una lista de tickets dispoibles
        self.current_ticket = 0  # posicion del primer ticket disponible

    def __repr__(self):
        return f'tickets disponibles: {sum(self.tickets)}\n'  # imprime la cantidad de tickets restantes


# Venta de t_sold tickets sin usar threads
def seller_func(t_list, t_sold):
    for i in range(t_sold):
        t_list.tickets[t_list.current_ticket] = 0  # ticket vendido pasa a tener valor 0
        t_list.current_ticket += 1  # se actualiza al siguiente ticket dispoible


# Vendedor de tickets, usa threads
class Seller(thr.Thread):
    def __init__(self, t_list, t_sold, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.t_list = t_list  # lista de tickets
        self.t_sold = t_sold  # cantidad de tickets que debe vender

    def sell(self):
        self.t_list.tickets[self.t_list.current_ticket] = 0  # al vender un ticket se cambia su valor en la lista
        self.t_list.current_ticket += 1  # se actualiza para el siguiente ticket disponible
    
    def run(self):  # Vende la cantidad asignada de tickets
        for i in range(self.t_sold):
            self.sell()

            
# Vendedor de tickets, pero con locks
class GoodSeller(thr.Thread):
    loki = thr.Lock()  # todos los vendedores comparten un mismo lock
    
    def __init__(self, t_list, t_sold, *args, **kwargs):  # mismo init
        super().__init__(*args, **kwargs)
        self.t_list = t_list
        self.t_sold = t_sold

    def sell(self):  # mismo vender
        self.t_list.tickets[self.t_list.current_ticket] = 0
        self.t_list.current_ticket += 1
    
    def run(self):
        for i in range(self.t_sold):
            # antes de vender, verifica que ningun otro vendedor esté vendiendo el mismo ticket
            with self.loki:  
                self.sell()
    

if __name__ == '__main__':
    print('\n _____________________________________________')
    print('|                                             |')
    print('|                Sin Threading                |')
    print('|_____________________________________________|\n')

    tickets = TicketList(100000)
    print(tickets)

    seller_func(tickets, 50000)
    seller_func(tickets, 50000)
    print(tickets)

    print('\n _____________________________________________')
    print('|                                             |')
    print('|                Con Threading                |')
    print('|_____________________________________________|\n')
    tickets = TicketList(1000000)
    print(tickets)

    seller1 = Seller(tickets, 500000)
    seller2 = Seller(tickets, 500000)

    seller1.start()
    seller2.start()
    seller1.join()
    seller2.join()

    print(tickets)



    print('\n _____________________________________________')
    print('|                                             |')
    print('|                  Con Locks                  |')
    print('|_____________________________________________|\n')

    tickets = TicketList(1000000)
    print(tickets)

    seller1 = GoodSeller(tickets, 500000)
    seller2 = GoodSeller(tickets, 500000)

    seller1.start()
    seller2.start()
    seller1.join()
    seller2.join()

    print(tickets)

### Aplicando lo anterior a nuesto juego:

In [ ]:
class Enemigo(thr.Thread):
    count_lock = thr.Lock()
    move_count = 0
    
    def __init__(self, nombre, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.nombre = nombre
        self.daemon = True
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}')
        
    def run(self):
        for i in range(3):
            self.mover()
            with self.count_lock:
                Enemigo.move_count += 1
            time.sleep(1)
            
# Jugador sin evento
class Jugador(thr.Thread):
    def __init__(self, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y

    def mover(self):
        mov = int(input('Seleccione un número de casillas para mover.'))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}\n')
        
    def run(self):
        for i in range(1):
            self.mover()
            
# Instanciamos
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

# Iniciamos
el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

# imprimimos la cantidad de movimientos que los enemigos lograron hacer cuando termina el jugador
jugador.join()
print(f'Movimientos: {Enemigo.move_count}\n')

# ¡Gracias por su atención!
### Recuerden llenar el Feedback :D